In [1]:
# Este código está basado el código del repositorio https://github.com/k-sys/covid-19
# Este repositorio fue clonado el 2020-04-25 en https://github.com/coronamex/covid-20
# Tiene pequeñas modificaciones para realizar estimaciones con los datos de México distribuidos
# a través de CoronaMex.
# El código tomado de https://github.com/k-sys/covid-19 está en el dominito público.
# El resto se distribuye con una licencia GPL-3

# (C) Copyright 2020 Sur Herrera Paredes

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.

# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

# For some reason Theano is unhappy when I run the GP, need to disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import requests
import pymc3 as pm
import pandas as pd
import numpy as np
import theano
import theano.tensor as tt

from matplotlib import pyplot as plt
from matplotlib import dates as mdates
from matplotlib import ticker

from datetime import date
from datetime import datetime

from IPython.display import clear_output

%config InlineBackend.figure_format = 'retina'

In [2]:
def confirmed_to_onset(confirmed, p_delay):

    assert not confirmed.isna().any()
    
    # Reverse cases so that we convolve into the past
    convolved = np.convolve(confirmed[::-1].values, p_delay)

    # Calculate the new date range
    dr = pd.date_range(end=confirmed.index[-1],
                       periods=len(convolved))

    # Flip the values and assign the date range
    onset = pd.Series(np.flip(convolved), index=dr)
    
    return onset

In [3]:
def adjust_onset_for_right_censorship(onset, p_delay):
    cumulative_p_delay = p_delay.cumsum()
    
    # Calculate the additional ones needed so shapes match
    ones_needed = len(onset) - len(cumulative_p_delay)
    padding_shape = (0, ones_needed)
    
    # Add ones and flip back
    cumulative_p_delay = np.pad(
        cumulative_p_delay,
        padding_shape,
        constant_values=1)
    cumulative_p_delay = np.flip(cumulative_p_delay)
    
    # Adjusts observed onset values to expected terminal onset values
    adjusted = onset / cumulative_p_delay
    
    return adjusted, cumulative_p_delay


In [4]:
class MCMCModel(object):
    
    def __init__(self, region, onset, cumulative_p_delay, window=50):
        
        # Just for identification purposes
        self.region = region
        
        # For the model, we'll only look at the last N
        self.onset = onset.iloc[-window:]
        self.cumulative_p_delay = cumulative_p_delay[-window:]
        
        # Where we store the results
        self.trace = None
        self.trace_index = self.onset.index[1:]

    def run(self, chains=1, tune=3000, draws=1000, target_accept=.95):

        with pm.Model() as model:

            # Random walk magnitude
            step_size = pm.HalfNormal('step_size', sigma=.03)

            # Theta random walk
            theta_raw_init = pm.Normal('theta_raw_init', 0.1, 0.1)
            theta_raw_steps = pm.Normal('theta_raw_steps', shape=len(self.onset)-2) * step_size
            theta_raw = tt.concatenate([[theta_raw_init], theta_raw_steps])
            theta = pm.Deterministic('theta', theta_raw.cumsum())

            # Let the serial interval be a random variable and calculate r_t
            serial_interval = pm.Gamma('serial_interval', alpha=6, beta=1.5)
            gamma = 1.0 / serial_interval
            r_t = pm.Deterministic('r_t', theta/gamma + 1)

            inferred_yesterday = self.onset.values[:-1] / self.cumulative_p_delay[:-1]
            
            expected_today = inferred_yesterday * self.cumulative_p_delay[1:] * pm.math.exp(theta)

            # Ensure cases stay above zero for poisson
            mu = pm.math.maximum(.1, expected_today)
            observed = self.onset.round().values[1:]
            cases = pm.Poisson('cases', mu=mu, observed=observed)

            self.trace = pm.sample(
                chains=chains,
                tune=tune,
                draws=draws,
                target_accept=target_accept)
            
            return self
    
    def run_gp(self):
        with pm.Model() as model:
            gp_shape = len(self.onset) - 1

            length_scale = pm.Gamma("length_scale", alpha=3, beta=.4)

            eta = .05
            cov_func = eta**2 * pm.gp.cov.ExpQuad(1, length_scale)

            gp = pm.gp.Latent(mean_func=pm.gp.mean.Constant(c=0), 
                              cov_func=cov_func)

            # Place a GP prior over the function f.
            theta = gp.prior("theta", X=np.arange(gp_shape)[:, None])

            # Let the serial interval be a random variable and calculate r_t
            serial_interval = pm.Gamma('serial_interval', alpha=6, beta=1.5)
            gamma = 1.0 / serial_interval
            r_t = pm.Deterministic('r_t', theta / gamma + 1)

            inferred_yesterday = self.onset.values[:-1] / self.cumulative_p_delay[:-1]
            expected_today = inferred_yesterday * self.cumulative_p_delay[1:] * pm.math.exp(theta)

            # Ensure cases stay above zero for poisson
            mu = pm.math.maximum(.1, expected_today)
            observed = self.onset.round().values[1:]
            cases = pm.Poisson('cases', mu=mu, observed=observed)

            self.trace = pm.sample(chains=1, tune=1000, draws=1000, target_accept=.8)
        return self

In [5]:
def df_from_model(model):
    
    r_t = model.trace['r_t']
    mean = np.mean(r_t, axis=0)
    median = np.median(r_t, axis=0)
    hpd_90 = pm.stats.hpd(r_t, credible_interval=.9)
    hpd_50 = pm.stats.hpd(r_t, credible_interval=.5)
    
    idx = pd.MultiIndex.from_product([
            [model.region],
            model.trace_index
        ], names=['region', 'date'])
        
    df = pd.DataFrame(data=np.c_[mean, median, hpd_90, hpd_50], index=idx,
                 columns=['mean', 'median', 'lower_90', 'upper_90', 'lower_50','upper_50'])
    return df

def create_and_run_model(name, state):
    confirmed = state.positive.diff().dropna()
    onset = confirmed_to_onset(confirmed, p_delay)
    adjusted, cumulative_p_delay = adjust_onset_for_right_censorship(onset, p_delay)
    return MCMCModel(name, onset, cumulative_p_delay).run()

In [6]:
def calcular_p_retraso(serie_sintomas, serie_confirmacion):
    """Calcular distribución en retraso entre día de inicio de síntomas
    y confirmación de un caso."""
    
    # serie_sintomas = "../../datos/datos_abiertos/serie_tiempo_nacional_confirmados.csv"
    # serie_confirmacion = "../../datos/datos_abiertos/serie_tiempo_nacional_fecha_confirmacion.csv"
    
    # Leer tablas
    Dat_sintomas = pd.read_csv(serie_sintomas)
    Dat_sintomas.head()
    Dat_confirmacion = pd.read_csv(serie_confirmacion)
    Dat_confirmacion.head()
    Dat_sintomas = Dat_sintomas[['fecha', 'sintomas_acumulados']]
    Dat_confirmacion = Dat_confirmacion[['fecha', 'casos_acumulados']]
    
    # Unir tablas
    Dat = pd.concat([Dat_sintomas.set_index('fecha'), Dat_confirmacion.set_index('fecha')], axis=1, sort=False).reset_index(col_fill = 'fecha')
    Dat.columns = ['fecha', 'sintomas_acumulados', 'casos_acumulados']
    Dat = Dat.fillna(0)
    # Dat['dif'] = Dat.sintomas_acumulados - Dat.casos_acumulados
    Dat.fecha = pd.to_datetime(Dat.fecha)
    # Dat
    
    # Preparar serie vacía
    dias = (pd.Series(max(Dat.fecha)) - pd.Series(min(Dat.fecha))).dt.days[0]
    p_retraso = pd.Series(np.zeros(dias))
    for i, fila in Dat.iterrows():
        for j in range(i, Dat.shape[0]):
            # print(i, j)
            if Dat.casos_acumulados[j] >= Dat.sintomas_acumulados[i]:
                # print("hola")
                # Sumar casos retrasados n dias
                p_retraso[j - i] = p_retraso[j - i] + abs(Dat.sintomas_acumulados[i] - Dat.casos_acumulados[j])
                break
    
    # Calcular prob
    # p_retraso.sum()
    p_retraso = p_retraso / p_retraso.sum()
    # print(p_retraso.to_string())
    
    # Limpiar
    ii = p_retraso > 0
    ii_max = np.max(p_retraso.index[ii])
    p_retraso = p_retraso[0:ii_max]

    return p_retraso

In [7]:
# url = 'https://covidtracking.com/api/v1/states/daily.csv'
# states = pd.read_csv(url,
#                      parse_dates=['date'],
#                      index_col=['state', 'date']).sort_index()

# # Note: GU/AS/VI do not have enough data for this model to run
# # Note: PR had -384 change recently in total count so unable to model
# states = states.drop(['MP', 'GU', 'AS', 'PR', 'VI'])

# ## SUR
# states = {state: grp for state, grp in states.groupby('state')}['CA']
# states.head()

In [8]:
# # Make sure that all the states have current data
# today = datetime.combine(date.today(), datetime.min.time())
# last_updated = states.reset_index('date').groupby('state')['date'].max()
# is_current = last_updated < today

# try:
#     assert is_current.sum() == 0
# except AssertionError:
#     print("Not all states have updated")
#     display(last_updated[is_current])

# # Ensure all case diffs are greater than zero
# for state, grp in states.groupby('state'):
#     new_cases = grp.positive.diff().dropna()
#     is_positive = new_cases.ge(0)
    
#     try:
#         assert is_positive.all()
#     except AssertionError:
#         print(f"Warning: {state} has date with negative case counts")
#         display(new_cases[~is_positive])
        
# # Let's make sure that states have added cases
# idx = pd.IndexSlice
# assert not states.loc[idx[:, '2020-04-22':'2020-04-23'], 'positive'].groupby('state').diff().dropna().eq(0).any()

In [9]:
# # Load the patient CSV
# patients = pd.read_csv(
#     '../../COVID-20/data/linelist.csv',
#     parse_dates=False,
#     usecols=[
#         'date_confirmation',
#         'date_onset_symptoms'],
#     low_memory=False)

# patients.columns = ['Onset', 'Confirmed']

# # There's an errant reversed date
# patients = patients.replace('01.31.2020', '31.01.2020')

# # Only keep if both values are present
# patients = patients.dropna()

# # Must have strings that look like individual dates
# # "2020.03.09" is 10 chars long
# is_ten_char = lambda x: x.str.len().eq(10)
# patients = patients[is_ten_char(patients.Confirmed) & 
#                     is_ten_char(patients.Onset)]

# # Convert both to datetimes
# patients.Confirmed = pd.to_datetime(
#     patients.Confirmed, format='%d.%m.%Y')
# patients.Onset = pd.to_datetime(
#     patients.Onset, format='%d.%m.%Y')

# # Only keep records where confirmed > onset
# patients = patients[patients.Confirmed >= patients.Onset]

# ## SUR
# patients.head()

In [10]:
# # Calculate the delta in days between onset and confirmation
# delay = (patients.Confirmed - patients.Onset).dt.days

# # Convert samples to an empirical distribution
# p_delay = delay.value_counts().sort_index()
# new_range = np.arange(0, p_delay.index.max()+1)
# p_delay = p_delay.reindex(new_range, fill_value=0)
# p_delay /= p_delay.sum()

# # Show our work
# fig, axes = plt.subplots(ncols=2, figsize=(9,3))
# p_delay.plot(title='P(Delay)', ax=axes[0])
# p_delay.cumsum().plot(title='P(Delay <= x)', ax=axes[1])
# for ax in axes:
#     ax.set_xlabel('days')

In [11]:
# state = 'CA'
# confirmed = states.xs(state).positive.diff().dropna()
# onset = confirmed_to_onset(confirmed, p_delay)
# ## SUR
# onset.head()

In [12]:
# adjusted, cumulative_p_delay = adjust_onset_for_right_censorship(onset, p_delay)

# ## SUR
# print(adjusted.head())
# cumulative_p_delay

In [13]:
# models = {}

# for state, grp in states.groupby('state'):
    
#     print(state)
    
#     if state in models:
#         print(f'Skipping {state}, already in cache')
#         continue
    
#     models[state] = create_and_run_model(state, grp.droplevel(0))

In [14]:
# name = 'CA'
# confirmed = states.positive.diff().dropna()
# # confirmed = state.positive.diff().dropna()
# confirmed = confirmed.droplevel(0)
# # confirmed
# onset = confirmed_to_onset(confirmed, p_delay)
# # confirmed_to_onset(confirmed, p_delay)
# adjusted, cumulative_p_delay = adjust_onset_for_right_censorship(onset, p_delay)
# # adjust_onset_for_right_censorship(onset, p_delay)
# m1 = MCMCModel(name, onset, cumulative_p_delay).run()
# m1

In [41]:
# Calcular p_retraso
# serie_sintomas_nacional = "../../datos/datos_abiertos/serie_tiempo_nacional_confirmados.csv"
# serie_confirmacion_nacional = "../../datos/datos_abiertos/serie_tiempo_nacional_fecha_confirmacion.csv"
# serie_confirmacion_estados = "../../datos/datos_abiertos/serie_tiempo_estados_fecha_confirmacion.csv"
# dir_estimado = "../estimados/"

serie_sintomas_nacional = "../datos/datos_abiertos/serie_tiempo_nacional_confirmados.csv"
serie_confirmacion_nacional = "../datos/datos_abiertos/serie_tiempo_nacional_fecha_confirmacion.csv"
serie_confirmacion_estados = "../datos/datos_abiertos/serie_tiempo_estados_fecha_confirmacion.csv"
dir_estimado = "estimados/"

print("Parámetros leídos")
p_retraso = calcular_p_retraso(serie_sintomas=serie_sintomas_nacional, serie_confirmacion=serie_confirmacion_nacional)

In [19]:
# sintomas = pd.read_csv(serie_sintomas)
# sintomas = sintomas[['fecha', 'sintomas_nuevos']]
# sintomas.columns = ['date', 'positives']
# sintomas = sintomas.set_index('date')
# sintomas

In [20]:
# # Leer serie de casos confirmados
# casos = pd.read_csv(serie_confirmacion_estados)
# casos = casos[['fecha', 'casos_nuevos']]
# casos.columns = ['date', 'positives']
# casos = casos.set_index('date')

In [21]:
Dat = pd.read_csv(serie_confirmacion_estados)
Dat.head()

,estado,fecha,casos_acumulados_um,casos_acumulados_res,muertes_acumuladas_um,muertes_acumuladas_res,casos_nuevos_um,casos_nuevos_res,muertes_nuevas_um,muertes_nuevas_res
0,Aguascalientes,2020-03-15,1,NaN,0,NaN,1,NaN,0,NaN
1,Aguascalientes,2020-03-16,1,NaN,0,NaN,0,NaN,0,NaN
2,Aguascalientes,2020-03-17,1,NaN,0,NaN,0,NaN,0,NaN
3,Aguascalientes,2020-03-18,1,NaN,0,NaN,0,NaN,0,NaN
4,Aguascalientes,2020-03-19,4,NaN,0,NaN,3,NaN,0,NaN


In [22]:
modelos = {}
for ent in Dat.estado.unique():
    casos = Dat[ Dat.estado == ent ]
    casos = casos[['fecha', 'casos_nuevos_um']]
    casos.columns = ['date', 'positives']
    casos = casos.set_index('date')
    
    if casos.positives.sum() > 500:
        print(ent)
        inicio = confirmed_to_onset(casos.positives, p_retraso)
        ajustados, p_retraso_acumulado = adjust_onset_for_right_censorship(inicio, p_retraso)
        ii = ajustados.isin([np.nan, np.inf,-np.inf])
        p_retraso_acumulado = p_retraso_acumulado[~ii]
        m1 = MCMCModel(ent, inicio[~ii], p_retraso_acumulado).run()
        
        modelos[ent] = m1

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [serial_interval, theta_raw_steps, theta_raw_init, step_size]
Sampling chain 0, 0 divergences: 100%|██████████| 4000/4000 [00:46<00:00, 86.54it/s] 
Only one chain was sampled, this makes it impossible to run some convergence checks
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [serial_interval, theta_raw_steps, theta_raw_init, step_size]
Sampling chain 0, 0 divergences: 100%|██████████| 4000/4000 [01:58<00:00, 33.85it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [serial_interval, theta_raw_steps, theta_raw_init, step_size]
Sampling chain 0, 0 divergences: 100%|██████████| 4000/4000 [00:43<00:00, 92.35it/s] 
Only one chain was samp

In [23]:
modelos

{'Baja California': <__main__.MCMCModel at 0x7f747a860550>,
 'Ciudad de México': <__main__.MCMCModel at 0x7f747a860b70>,
 'México': <__main__.MCMCModel at 0x7f7479c0cda0>,
 'Quintana Roo': <__main__.MCMCModel at 0x7f7479fe8860>,
 'Sinaloa': <__main__.MCMCModel at 0x7f746a640780>,
 'Tabasco': <__main__.MCMCModel at 0x7f7465c8f0f0>}

In [48]:
# Combinar tablas
Tab = pd.DataFrame()
for k in modelos.keys():
    print(k)
    m1 = modelos[k]
    tab = df_from_model(m1)
    tab = tab.droplevel(0)
    tab = tab.reset_index()
    tab['estado'] = k
    
    Tab = pd.concat([Tab, tab])
Tab['fecha_estimado'] = date.today().isoformat()

Baja California
Ciudad de México
México
Quintana Roo
Sinaloa
Tabasco


In [51]:
archivo = dir_estimado + '/rt_live_estimados.csv'
Tab.to_csv(archivo, index=False)

In [ ]:
# inicio = confirmed_to_onset(casos.positives, p_retraso)
# # inicio = sintomas.positives
# ajustados, p_retraso_acumulado = adjust_onset_for_right_censorship(inicio, p_retraso)
# # ajustados
# ii = ajustados.isin([np.nan, np.inf,-np.inf])
# p_retraso_acumulado = p_retraso_acumulado[~ii]
# m1 = MCMCModel('MX', inicio[~ii], p_retraso_acumulado).run()

In [ ]:
# results = df_from_model(m1)
# results = results.droplevel(0)
# results['mean']
# results = results.reset_index()
# results.head()

In [ ]:
# plt.plot_date(results['date'], results['median'], linestyle = "-", lw=2,markevery=1)
# plt.fill_between(results.date,results['lower_90'].values, results['upper_90'].values)

In [ ]:
#  ax.plot(result['median'],
#             marker='o',
#             markersize=4,
#             markerfacecolor='w',
#             lw=1,
#             c=c,
#             markevery=2)
#     ax.fill_between(
#         result.index,
#         result['lower_90'].values,
#         result['upper_90'].values,
#         color=ci,
#         lw=0)

# plt.plot(results['median'],
#         marker='o',
#             markersize=4,
#             markerfacecolor='w',
#             lw=1,
#             markevery=2)
# plt.fill_between(results.index,results['lower_90'].values, results['upper_90'].values)
# # plt.show()

In [ ]:
# plt.fill_between?

In [ ]:
# results.index('date')